# **QA**

In [ ]:
# import libraries
import uproot
import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit
from scipy.stats import norm

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

## **I. Initial Checks**

### **1. Injected Anti-Sexaquarks**

In [ ]:
# plot pt, phi and rapidity of the injected anti-sexaquarks
file = uproot.open("../macros/test/InjectedResults.root")
tree = file["IC"]

# convert branch "SM" into a numpy array
SM = tree["SM"].array() # mass
SPx = tree["SPx"].array() # momentum
SPy = tree["SPy"].array()
SPz = tree["SPz"].array()

SPt = np.sqrt(SPx**2 + SPy**2)
SPhi = np.arctan2(SPy, SPx)
SPhi = np.where(SPhi < 0, SPhi + 2 * np.pi, SPhi)
SMt = np.sqrt(SM**2 + SPt**2)
SRapidity = np.arcsinh(SPz / SMt)

variables = [SPt, SPhi, SRapidity]
variable_names = [r"$p_T$ (GeV/c)", r"$\phi$ (rad)", "Rapidity"]

plt.figure(figsize=(6 * len(variables), 6))

for i in range(len(variables)):

    ax = plt.subplot(1, len(variables), i + 1)

    n, bins, patches = ax.hist(variables[i], bins=100, color=colors[i], histtype="step", label=r"MC Gen. Injected $\bar{S}$", linewidth=2)

    n_entries = len(variables[i])
    stats = f"Entries: {n_entries:.0f}"
    ax.hist(variables[i], bins=100, alpha=0., label=stats) # invisible histogram to show stats on the legend

    ax.set_ylim(top=1.2*np.max(n))
    ax.set_ylabel("Counts", loc='top', fontsize=12, fontweight='bold')
    ax.set_xlabel(variable_names[i], loc='right', fontsize=12)
    ax.tick_params(direction='in', top=True, right=True)
    ax.grid(True, linestyle='--', alpha=0.5)

    plt.legend(fontsize=12)

plt.tight_layout()
plt.savefig("QA_InjectedSexaquarks.svg")
plt.show()

### **2. Struck Nucleon**

In [ ]:
# plot p, phi, theta, px, py, pz of struck nucleons
def gaussian(x, a, mu, sigma):
    return a * np.exp(-0.5 * ((x - mu) / sigma)**2)

file = uproot.open("../macros/test/InjectedResults.root")
# file = uproot.open("../macros/SimpleGen_FermiMotion.root")
tree = file["IC"]

# convert branch "SM" into a numpy array
NPx = tree["NPx"].array() # momentum
NPy = tree["NPy"].array()
NPz = tree["NPz"].array()

NP = np.sqrt(NPx**2 + NPy**2 + NPz**2)
NPt = np.sqrt(NPx**2 + NPy**2)
NPhi = np.arctan2(NPy, NPx)
NPhi = np.where(NPhi < 0, NPhi + 2 * np.pi, NPhi)
NTheta = np.arccos(NPz / NP)

variables = [NP, NPhi, NTheta, NPx, NPy, NPz]
variable_names = [r"$p$ (GeV/c)", r"$\phi$ (rad)", r"$\theta$ (rad)", r"$p_x$ (GeV/c)", r"$p_y$ (GeV/c)", r"$p_z$ (GeV/c)"]

n = len(variables)
rows = n // 3 + (n % 3 > 0)  # Calculate the number of rows needed
cols = 3  # Set the number of columns

plt.figure(figsize=(6 * cols, 6 * rows))

for i in range(n):

    row = i // cols
    col = i % cols
    ax = plt.subplot(rows, cols, row * cols + col + 1)

    bin_content, bins, patches = ax.hist(variables[i], bins=100, color=colors[i], histtype="step", label=r"MC Gen. Struck Nucleon", linewidth=2)

    n_entries = len(variables[i])
    stats = f"Entries: {n_entries:.0f}"
    ax.hist(variables[i], bins=100, alpha=0., label=stats) # invisible histogram to show stats on the legend

    # if the var is momentum, fit a gaussian to it
    if variable_names[i] == r"$p$ (GeV/c)":
        avg = np.mean(variables[i])
        std = np.std(variables[i])
        popt, _ = curve_fit(gaussian, bins[:-1], bin_content, p0=[1, avg, std])
        x = np.linspace(np.min(variables[i]), np.max(variables[i]), 1000)
        ax.plot(x, gaussian(x, *popt), color=colors[3], linestyle="-", label=r"Gaussian: $\mu$ = %.3f, $\sigma$ = %.3f" % (popt[1], popt[2]), linewidth=2)

    ax.set_ylim(top=1.3 * np.max(bin_content))
    ax.set_ylabel("Counts", loc='top', fontsize=12, fontweight='bold')
    ax.set_xlabel(variable_names[i], loc='right', fontsize=12)
    ax.tick_params(direction='in', top=True, right=True)
    ax.grid(True, linestyle='--', alpha=0.5)

    plt.legend(fontsize=12)

plt.tight_layout()
plt.savefig("QA_StruckNucleons_A.svg")
# plt.savefig("QA_StruckNucleons_B.svg")
plt.show()

### **3. Anti-Sexaquark—Nucleon Interaction**

In [ ]:
# plot radius of signal interactions
file = uproot.open("../sexaquark/test_sig/AnalysisResults_A1.8_kalmanV0s.root")

histograms = file["Hists/"]
hist_dict = {hist.name: hist for hist in histograms}

plt.figure(figsize=(6, 6))

hist_name = "MCGen_All_AntiSexaquark_Radius"
values, edges = hist_dict[hist_name].to_numpy()

plot_name = "MC Gen. Signal Interaction"
plt.hist(edges[:-1], edges, weights=values, label=plot_name, histtype='step', color=colors[7], linewidth=2)

n_entries = np.sum(values)
stats = f"Entries: {n_entries:.0f}"
plt.hist(edges[:-1], edges, alpha=0., label=stats) # invisible histogram to show stats on the legend

plt.ylabel('Counts', loc='top', fontsize=12, fontweight='bold')
plt.xlabel('Radius (cm)', loc='right', fontsize=12)
plt.ylim(top=1.3 * np.max(values))
plt.tick_params(direction='in', top=True, right=True)
plt.grid(True, linestyle='--', alpha=0.5)

# add vertical lines on 5 and 180
plt.axvline(x=5, color=colors[3], alpha=0.8, linestyle="--", linewidth=2)
plt.axvline(x=180, color=colors[3], alpha=0.8, linestyle="--", linewidth=2)

plt.legend(loc="upper center", fontsize=12)

plt.tight_layout()
plt.savefig("QA_SignalInteraction_Radius.svg")
plt.show()

In [ ]:
# plot pt, phi and rapidity of the injected anti-sexaquarks (after interaction)

variables = ["Mass", "Pt", "Pz", "Rapidity"] # TEMPORARY: change Pz by Phi!
variable_names = [r"Mass (GeV/$c^2$)", r"$p_T$ (GeV/c)", r"$\phi$ (rad)", "Rapidity"]

n = len(variables)
cols = 2
rows = 2
plt.figure(figsize=(6 * cols, 6 * rows))

for i in range(len(variables)):

    row = i // cols
    col = i % cols
    plt.subplot(rows, cols, row * cols + col + 1)

    hist_name = f"MCGen_All_AntiSexaquark_{variables[i]}"
    values, edges = hist_dict[hist_name].to_numpy()

    plot_name = r"MC Gen. $\bar{S}$ (after interaction)"
    plt.hist(edges[:-1], edges, weights=values, label=plot_name, histtype='step', color=colors[i], linewidth=2)

    n_entries = np.sum(values)
    stats = f"Entries: {n_entries:.0f}"
    plt.hist(edges[:-1], edges, weights=values, alpha=0., label=stats) # invisible histogram to show stats on the legend

    plt.ylim(top=1.2*np.max(values))
    plt.ylabel("Counts", loc='top', fontsize=12, fontweight='bold')
    plt.xlabel(variable_names[i], loc='right', fontsize=12)
    plt.tick_params(direction='in', top=True, right=True)
    plt.grid(True, linestyle='--', alpha=0.5)

    plt.legend(loc="upper left", fontsize=12)

plt.tight_layout()
plt.savefig("QA_SexaquarksAfterInteraction.svg")
plt.show()